In [69]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_1samp
import matplotlib.pyplot as plt
import warnings
from scipy.interpolate import PchipInterpolator
warnings.filterwarnings("ignore")

In [70]:
data_path = '../data/monthly/monthly.csv'
monthly_raw_data = pd.read_csv(data_path)

monthly_raw_data['RETX'] = pd.to_numeric(monthly_raw_data['RETX'], errors='coerce')
monthly_raw_data['ewretd'] = pd.to_numeric(monthly_raw_data['ewretd'], errors='coerce')

monthly_raw_data['RETX'] = monthly_raw_data['RETX'] - monthly_raw_data['ewretd']

cleaned_data = monthly_raw_data.dropna(subset=['RETX'])
cleaned_data = cleaned_data[~cleaned_data['RETX'].isin(['B','C'])]

cleaned_data['date'] = pd.to_datetime(cleaned_data['date'])

stocks_with_enough_data = cleaned_data.groupby('PERMNO').filter(lambda x: len(x) >= 85)

stocks_with_enough_data = stocks_with_enough_data.sort_values(['PERMNO','date']).reset_index(drop=True)

In [71]:
def calculate_cumulative_excess_returns(group,windowsize):
    """
    对每只股票按日期顺序,按原文应为加和而非乘积
    """
    # group 已按日期排序
    group['cum_excess_return_'+ str(windowsize)] = group['RETX'].rolling(window=windowsize, min_periods=12).sum()
    return group

stocks_with_enough_data.reset_index(drop=True)

for windowsize in [12,24,36,60]:
    stocks_with_enough_data = stocks_with_enough_data.groupby('PERMNO',group_keys=False).apply(calculate_cumulative_excess_returns,windowsize=windowsize)

In [72]:
# 定义起始和结束日期
start_date = pd.Timestamp('1931-01-01')
end_date = pd.Timestamp('1983-01-01')

# 49个1年的周期 (每年1月形成新的组合)
one_year_periods = pd.date_range(start=start_date, end=end_date, freq='AS')[:49]

# 10个5年的周期（每5年形成一次）
five_year_periods = [start_date + pd.DateOffset(years=5 * i) for i in range(10)]

# 16个3年的周期（每3年形成一次）
three_year_periods = [start_date + pd.DateOffset(years=3 * i) for i in range(16)]

# 24个2年的周期（1933年到1979年，只包含奇数年份）
two_year_odd_periods = [pd.Timestamp(f"{1933 + 2 * i}-01-01") for i in range(24)]

# 25个2年的周期（1932年到1980年，只包含偶数年份）
two_year_even_periods = [pd.Timestamp(f"{1932 + 2 * i}-01-01") for i in range(25)]

# 24个2年的周期 (deciles, 奇数年份)
two_year_deciles_odd_periods = [pd.Timestamp(f"{1933 + 2 * i}-01-01") for i in range(24)]

# 25个2年的周期 (deciles, 偶数年份)
two_year_deciles_even_periods = [pd.Timestamp(f"{1932 + 2 * i}-01-01") for i in range(25)]

# 合并所有的formation periods列表
all_formation_periods = (
    one_year_periods.tolist() +
    five_year_periods +
    three_year_periods +
    two_year_odd_periods +
    two_year_even_periods +
    two_year_deciles_odd_periods +
    two_year_deciles_even_periods
)

# 将所有日期排序
all_formation_periods = sorted(all_formation_periods)

In [73]:
five_year_periods = [pd.Timestamp(f"{1930 + 5 * i}-01-01") for i in range(10)]
three_year_periods = [pd.Timestamp(f"{1930 + 3 * i}-01-01") for i in range(16)]
two_year_odd_periods_35 = [pd.Timestamp(f"{1933 + 2 * i}-01-01") for i in range(24)]
two_year_even_periods_35 = [pd.Timestamp(f"{1932 + 2 * i}-01-01") for i in range(25)]
two_year_odd_periods_82 = [pd.Timestamp(f"{1933 + 2 * i}-01-01") for i in range(24)]
two_year_even_periods_82 = [pd.Timestamp(f"{1932 + 2 * i}-01-01") for i in range(25)]
one_year_periods = [pd.Timestamp(f"{1931 + i}-01-01") for i in range(49)]

portfolio_starts = {
    "10 five-year periods": (five_year_periods, 50),
    "16 three-year periods": (three_year_periods, 35),
    "24 two-year odd periods (35 stocks)": (two_year_odd_periods_35, 35),
    "25 two-year even periods (35 stocks)": (two_year_even_periods_35, 35),
    "24 two-year odd periods (82 stocks)": (two_year_odd_periods_82, 82),
    "25 two-year even periods (82 stocks)": (two_year_even_periods_82, 82),
    "49 one-year periods": (one_year_periods, 35)
}

selected_months = [1, 12, 13, 18, 24, 25, 36, 60]

all_results = []

for period_name, (periods, num_stocks) in portfolio_starts.items():
    all_periods = []
    for start_dt in periods:
        formation_date = start_dt - pd.offsets.MonthEnd(1)
        if "five-year" in period_name:
            lookback_start = formation_date - pd.DateOffset(months=59)
        elif "three-year" in period_name:
            lookback_start = formation_date - pd.DateOffset(months=35)
        elif "two-year" in period_name:
            lookback_start = formation_date - pd.DateOffset(months=23)
        else:
            lookback_start = formation_date - pd.DateOffset(months=11)

        lookback_end = formation_date
        lookback_df = stocks_with_enough_data[
            (stocks_with_enough_data['date'] >= lookback_start) &
            (stocks_with_enough_data['date'] <= lookback_end)
        ].copy()
        if lookback_df.empty:
            continue

        sub = lookback_df[lookback_df['date'] <= formation_date]
        portfolio_data = sub.groupby('PERMNO', group_keys=False).tail(1).copy()

        if "five-year" in period_name:
            portfolio_data = portfolio_data.sort_values('cum_excess_return_60', ascending=False)
        elif "three-year" in period_name:
            portfolio_data = portfolio_data.sort_values('cum_excess_return_36', ascending=False)
        elif "two-year" in period_name:
            portfolio_data = portfolio_data.sort_values('cum_excess_return_24', ascending=False)
        else:
            portfolio_data = portfolio_data.sort_values('cum_excess_return_12', ascending=False)
        winner_ids = portfolio_data.head(num_stocks)['PERMNO']
        loser_ids = portfolio_data.tail(num_stocks)['PERMNO']

        hold_start = start_dt
        if "five-year" in period_name:
            hold_end = start_dt + pd.DateOffset(months=60) - pd.Timedelta(days=1)
        elif "three-year" in period_name:
            hold_end = start_dt + pd.DateOffset(months=36) - pd.Timedelta(days=1)
        elif "two-year" in period_name:
            hold_end = start_dt + pd.DateOffset(months=24) - pd.Timedelta(days=1)
        else:
            hold_end = start_dt + pd.DateOffset(months=24) - pd.Timedelta(days=1)

        w_hold = stocks_with_enough_data[
            (stocks_with_enough_data['PERMNO'].isin(winner_ids)) &
            (stocks_with_enough_data['date'] >= hold_start) &
            (stocks_with_enough_data['date'] <= hold_end)
        ].copy()

        l_hold = stocks_with_enough_data[
            (stocks_with_enough_data['PERMNO'].isin(loser_ids)) &
            (stocks_with_enough_data['date'] >= hold_start) &
            (stocks_with_enough_data['date'] <= hold_end)
        ].copy()

        w_monthly = w_hold.groupby('date')['RETX'].mean().reset_index(name='avg_u_w')
        l_monthly = l_hold.groupby('date')['RETX'].mean().reset_index(name='avg_u_l')

        merged = pd.merge(w_monthly, l_monthly, on='date', how='inner')
        merged['CAR_w'] = merged['avg_u_w'].cumsum(skipna=True)
        merged['CAR_l'] = merged['avg_u_l'].cumsum(skipna=True)
        merged['test_period_start'] = start_dt

        all_periods.append(merged)

    all_periods_df = pd.concat(all_periods, ignore_index=True)

    winner_portfolio = all_periods_df['CAR_w'].mean()
    loser_portfolio = all_periods_df['CAR_l'].mean()
    difference_car = loser_portfolio - winner_portfolio

    all_periods_df['monthNum'] = all_periods_df.groupby('test_period_start')['date'].rank(method='first', ascending=True)

    t_statistics = {}
    for t_ in selected_months:
        sub = all_periods_df[all_periods_df['monthNum'] == t_]
        w_vals = sub['CAR_w']
        l_vals = sub['CAR_l']
        difference = l_vals - w_vals
        if len(w_vals) > 1 and len(l_vals) > 1:
            t_stat, _ = ttest_1samp(difference, 0)
        else:
            t_stat = np.nan
        t_statistics[f"T-Statistic (Month {t_})"] = t_stat

    final_table = {
        "Length of Formation Period": period_name,
        "Average No. of Stocks": num_stocks,
        "Winner Portfolio": winner_portfolio,
        "Loser Portfolio": loser_portfolio,
        "Difference in CAR": difference_car,
        **t_statistics
    }
    all_results.append(final_table)

final_df = pd.DataFrame(all_results)


In [74]:
print(final_df.to_string(index=False))

          Length of Formation Period  Average No. of Stocks  Winner Portfolio  Loser Portfolio  Difference in CAR  T-Statistic (Month 1)  T-Statistic (Month 12)  T-Statistic (Month 13)  T-Statistic (Month 18)  T-Statistic (Month 24)  T-Statistic (Month 25)  T-Statistic (Month 36)  T-Statistic (Month 60)
                10 five-year periods                     50         -0.205577        -0.052608           0.152970               1.894190                0.389454                0.600272                0.856986                1.091914                1.145396                1.569319                1.675734
               16 three-year periods                     35         -0.080990         0.022733           0.103722               1.749817                0.362552                0.599934                1.111790                2.048089                2.022007                1.972995                     NaN
 24 two-year odd periods (35 stocks)                     35         -0.003637        